In [1]:
import time
import bluetooth
from mindwavemobile.MindwaveDataPoints import RawDataPoint
from mindwavemobile.MindwaveDataPoints import *
from mindwavemobile.MindwaveDataPointReader import MindwaveDataPointReader
import textwrap
import pandas as pd
from datetime import datetime
import os.path

In [37]:
eeg_df = None
limit_of_data_taken  = 250
limit_of_ignore = 120
user_id = 59
user_activity_id = 8
user_mental_state_id = 5

In [3]:

def collect_data(user_id,user_activity_id,user_mental_state_id,session_id):
    
    mindwaveDataPointReader = MindwaveDataPointReader()
    mindwaveDataPointReader.start()
    all_rows = []
    if (mindwaveDataPointReader.isConnected()):    
        count = 0
        att = False
        med = False
        eeg = False
        
        current_row = {}
        count_ignore = 0
        while(True):
            
            if(count > limit_of_data_taken):
                break
            
            dataPoint = mindwaveDataPointReader.readNextDataPoint()
            
            if(count_ignore < limit_of_ignore):
                count_ignore += 1
                continue
            
            if (dataPoint.__class__ is RawDataPoint and dataPoint.__class__ is PoorSignalLevelDataPoint):
                continue
            elif(dataPoint.__class__ is AttentionDataPoint):
                if(att == False):
                    att = True
                    current_row['attention'] = dataPoint.attentionValue
                    #print(current_row['attention'])
            
            elif(dataPoint.__class__ is MeditationDataPoint):
                if(med == False):
                    current_row['meditation'] = dataPoint.meditationValue
                    #print(current_row['meditation'])
                    med = True
            elif(dataPoint.__class__ is EEGPowersDataPoint):
                if(eeg == False):
                    current_row['delta'] = dataPoint.delta
                    current_row['theta'] = dataPoint.theta
                    current_row['lowAlpha'] = dataPoint.lowAlpha
                    current_row['highAlpha'] = dataPoint.highAlpha
                    current_row['lowBeta'] = dataPoint.lowBeta
                    current_row['highBeta'] = dataPoint.highBeta
                    current_row['lowGamma'] = dataPoint.lowGamma
                    current_row['midGamma'] = dataPoint.midGamma
                    eeg = True
            
            if(att == True and med == True and eeg == True):
                att = False
                med = False
                eeg = False
                count += 1
                
                if(count % 10 == 0):
                    print("iteration: ",count)
                    
                current_row['recording_time'] = datetime.now()
                current_row['user_id'] = user_id
                current_row['user_activity_id'] = user_activity_id
                current_row['user_mental_state_id'] = user_mental_state_id
                current_row['session_id'] = session_id
                
                all_rows.append(current_row)
                #print(current_row)
                current_row = {}
       
                
    else:
        print(textwrap.dedent("""\
            Exiting because the program could not connect
            to the Mindwave Mobile device.""").replace("\n", " "))
                            
            
    #print(all_rows)
    return all_rows 
        


In [4]:
def reload_eeg_dataframe():
    #first, we will check whether there exists a file named eeg_dataset.csv file. If there is, then it will be loaded as df.
    #Otherwise, it will be created brand new.
    if(os.path.exists("eeg_dataset")):
        print("file exists")
  
        eeg_df = pd.read_csv("eeg_dataset",parse_dates = True)
        eeg_df['recording_time'] = eeg_df['recording_time'].astype("datetime64[ms]")
        session_id = eeg_df.iloc[-1]['session_id'] + 1
    else:
        print("file does not exist")
        eeg_df = pd.DataFrame(columns=['user_id','user_activity_id','user_mental_state_id','attention','meditation','delta','theta','lowAlpha','highAlpha','lowBeta','highBeta','lowGamma','midGamma','recording_time','session_id'])
        for cur_col in eeg_df.columns:

            if(cur_col == 'recording_time'):
                eeg_df[cur_col] = eeg_df[cur_col].astype("datetime64[ms]")
            elif(cur_col == 'user_activity_id' or cur_col=='user_id' or cur_col == 'user_mental_state_id' or cur_col == 'session_id' ):
                eeg_df[cur_col] = eeg_df[cur_col].astype("int") 
            else:
                eeg_df[cur_col] = eeg_df[cur_col].astype("float32") 
        session_id = 0
    return eeg_df,session_id

In [38]:
eeg_df,session_id = reload_eeg_dataframe()

all_rows = collect_data(user_id,user_activity_id,user_mental_state_id,session_id)
temp_df = pd.DataFrame(all_rows)
eeg_df = eeg_df.append([temp_df])
#eeg_df.dtypes
eeg_df.to_csv("eeg_dataset", encoding='utf-8', index=False)
print("done")

file exists
Discovered Mindwave Mobile...
iteration:  10
iteration:  20
iteration:  30
iteration:  40
iteration:  50
iteration:  60
iteration:  70
iteration:  80
iteration:  90
iteration:  100
iteration:  110
iteration:  120
iteration:  130
iteration:  140
iteration:  150
iteration:  160
iteration:  170
iteration:  180
iteration:  190
iteration:  200
iteration:  210
iteration:  220
iteration:  230
iteration:  240
iteration:  250
done


###### eeg_df.columns


In [52]:

if __name__ == '__main__':
    mindwaveDataPointReader = MindwaveDataPointReader()
    mindwaveDataPointReader.start()
    if (mindwaveDataPointReader.isConnected()):    
        while(True):
            dataPoint = mindwaveDataPointReader.readNextDataPoint()
            if (not dataPoint.__class__ is RawDataPoint):
                print(dataPoint)
				
    else:
        print(textwrap.dedent("""\
            Exiting because the program could not connect
            to the Mindwave Mobile device.""").replace("\n", " "))
        

Discovered Mindwave Mobile...
Meditation Level: 0
Attention Level: 0
EEG Powers:
                delta: 56
                theta: 2
                lowAlpha: 0
                highAlpha: 0
                lowBeta: 0
                highBeta: 1
                lowGamma: 0
                midGamma: 0
                
Poor Signal Level: 25
Meditation Level: 0
Attention Level: 0
EEG Powers:
                delta: 2372111
                theta: 255585
                lowAlpha: 32691
                highAlpha: 54146
                lowBeta: 68034
                highBeta: 76377
                lowGamma: 7438
                midGamma: 33045
                
Poor Signal Level: 25
Meditation Level: 0
Attention Level: 0
EEG Powers:
                delta: 1760724
                theta: 146555
                lowAlpha: 113680
                highAlpha: 126129
                lowBeta: 38410
                highBeta: 46443
                lowGamma: 18685
                midGamma: 146801
            

KeyboardInterrupt: 

In [26]:
def correct_data():
    eeg_df = pd.read_csv("eeg_dataset",parse_dates = True)
    eeg_df['recording_time'] = eeg_df['recording_time'].astype("datetime64[ms]")
    #eeg_df[eeg_df['session_id'] == 41]['user_id'] = 23
   
    #eeg_df.loc[eeg_df['session_id'] == 41, 'user_id'] = 23
    #eeg_df.loc[eeg_df['session_id'] == 58, 'user_id'] = 39
    print(eeg_df[eeg_df['session_id'] == 41])
    print(eeg_df[eeg_df['session_id'] == 58])
    #eeg_df.to_csv("eeg_dataset", encoding='utf-8', index=False)

In [27]:
correct_data()

       attention      delta  highAlpha  highBeta  lowAlpha  lowBeta  lowGamma  \
17391        0.0        0.0        0.0       0.0       0.0      0.0       0.0   
17392        0.0       67.0        1.0       0.0       2.0      0.0       0.0   
17393        0.0   416431.0    44293.0   43264.0   69323.0  18978.0   14587.0   
17394        0.0   658310.0     3365.0    4071.0   15728.0   1984.0    5141.0   
17395        0.0   364715.0    25055.0   14040.0   23238.0  13941.0   23885.0   
17396        0.0  1191049.0    17307.0    9239.0    3169.0   4872.0    3059.0   
17397       30.0  1505428.0    24959.0   15967.0   39297.0  12110.0    7813.0   
17398       23.0  1232959.0    40644.0   11481.0   63438.0  15182.0   18672.0   
17399       38.0   312640.0    20053.0   45754.0   52435.0  14500.0   23843.0   
17400       48.0   651961.0     3684.0   10624.0    6315.0  13103.0    2202.0   
17401       66.0    73539.0     9016.0    8702.0    7210.0   8822.0    2102.0   
17402       91.0   316550.0 